#### Importing all the required **Python** and **R** libraries 

In [ ]:
import os
import pathlib
import json
import pandas as pd
import numpy as np
import scanpy as sc
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")
import anndata as ad

%load_ext autoreload
%autoreload 2
#%load_ext lab_black

In [ ]:
metadata = pd.read_csv('../../../supplementary_data/mouse/atlas_metadata_inhouse.csv', delimiter=';', header=0)

In [ ]:
metadata = metadata[metadata['larry'] == 'no']

In [ ]:
metadata = metadata.set_index('sample_ID')


In [ ]:
metadata.fillna('', inplace = True)

In [ ]:
metadata

## read GEP

In [ ]:
adatas = []
batches = []
for sample in metadata.index.values:
    try:
        meta = metadata
        ad_ = sc.read_10x_h5(meta.loc[sample, 'path'])
        ad_.var_names_make_unique()
        ad_.obs['donor_id'] = [sample] * ad_.n_obs if meta.loc[sample,'mouse_ID'] == '' else [meta.loc[sample,'mouse_ID']] * ad_.n_obs   
        ad_.obs['disease'] = ['PDAC'] * ad_.n_obs
        ad_.obs['model'] = [meta.loc[sample, 'model']] * ad_.n_obs
        ad_.obs['barcoded'] = [meta.loc[sample, 'larry']] * ad_.n_obs
        ad_.obs['sex'] = [meta.loc[sample, 'sex']] * ad_.n_obs
        ad_.obs['strain'] = [meta.loc[sample, 'strain']] * ad_.n_obs
        ad_.obs['genotype'] = [meta.loc[sample, 'genotype group']] * ad_.n_obs
        ad_.obs['treatment'] = [meta.loc[sample, 'treatment']] * ad_.n_obs
        ad_.obs['cell_filtering'] = [meta.loc[sample, 'enriched fraction']] * ad_.n_obs
        adatas.append(ad_)
        batches.append(sample)
    except Exception as e:
        print(f"Error processing sample {sample}: {e}")

In [ ]:
adata = adatas[0].concatenate(adatas[1:], join='outer', batch_key = 'sampleID', batch_categories = batches)

In [ ]:
adata.var = adata.var.iloc[:, [0]]

In [ ]:
for key in adata.obs.columns:
    adata.obs[key] = adata.obs[key].astype(str).astype('category')

In [ ]:
adata.write('/mnt/storage/Daniele/atlases/mouse/01_mouse_no_larry_raw.h5ad')